# POST an EAMENA dataset on Zenodo

Post an EAMENA dataset on Zenodo from an EAMENA Search URL (ex: Sistan dataset, caravanserais, etc.)

---

documentation: https://developers.zenodo.org/#quickstart-upload

## Libraries

In [1]:
import os
import requests
import json
import zipfile
import pandas as pd
import numpy as np

# needed to export as JSON
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

## Your dataset

### Variables/metadata fields


Search URL. This GeoJSON url comes from a Search query on HP (example: [Sistan](https://github.com/eamena-project/eamena-arches-dev/blob/main/projects/sistan/README.md#dataset)).

---

documentation: [GeoJSON URL](https://github.com/eamena-project/eamenaR/blob/ed96039aa9e98697311b9bbdf5eaf3f6e0c36597/README.md#exported-files)

In [2]:
GEOJSON_URL = r"https://database.eamena.org/api/search/export_results?paging-filter=1&tiles=true&format=geojson&reportlink=false&precision=6&total=326&language=en&advanced-search=%5B%7B%22op%22%3A%22and%22%2C%2234cfea78-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22~%22%2C%22lang%22%3A%22en%22%2C%22val%22%3A%22Sistan%22%7D%2C%2234cfea87-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%7D%2C%7B%22op%22%3A%22or%22%2C%2234cfea69-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%2C%2234cfea73-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%2C%2234cfea43-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%224ed99706-2d90-449a-9a70-700fc5326fb1%22%7D%2C%2234cfea5d-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22%22%2C%22val%22%3A%22%22%7D%2C%2234cfea95-c2c0-11ea-9026-02e7594ce0a0%22%3A%7B%22op%22%3A%22~%22%2C%22lang%22%3A%22en%22%2C%22val%22%3A%22%22%7D%7D%5D&resource-type-filter=%5B%7B%22graphid%22%3A%2234cfe98e-c2c0-11ea-9026-02e7594ce0a0%22%2C%22name%22%3A%22Heritage%20Place%22%2C%22inverted%22%3Afalse%7D%5D&map-filter=%7B%22type%22%3A%22FeatureCollection%22%2C%22features%22%3A%5B%7B%22id%22%3A%22e84886109295dcb2d515f9ab792832bf%22%2C%22type%22%3A%22Feature%22%2C%22properties%22%3A%7B%22buffer%22%3A%7B%22width%22%3A10%2C%22unit%22%3A%22m%22%7D%2C%22inverted%22%3Afalse%7D%2C%22geometry%22%3A%7B%22coordinates%22%3A%5B%5B%5B61.5629662657594%2C31.341070427554456%5D%2C%5B61.39269902363566%2C31.226740239181964%5D%2C%5B61.52316353383432%2C30.977760218239922%5D%2C%5B61.773036239808164%2C30.92940344148805%5D%2C%5B61.89244443558445%2C31.037461248216815%5D%2C%5B61.933352798951745%2C31.22484931983834%5D%2C%5B61.5629662657594%2C31.341070427554456%5D%5D%5D%2C%22type%22%3A%22Polygon%22%7D%7D%5D%7D"

Adapt these metadata to your dataset

In [29]:
FILENAME = "data5" # this is also the title in metadata
UPLOAD_TYPE = 'dataset'
DESCRIPTION = 'This is my first upload'
CREATORS_NAMES = "EAMENA database"
CREATORS_AFFILIATION = "University of Oxford, University of Southampton"
LICENSE = 'cc-by'
KEYWORDS = ['Sistan','EAMENA']
# gather in a dictionary
metadata = {
     'metadata': {
         'title': FILENAME,
         'upload_type': UPLOAD_TYPE,
         'description': DESCRIPTION,
         'creators': [{'name': CREATORS_NAMES,
                       'affiliation': CREATORS_AFFILIATION}],
         'license': LICENSE,
         'keywords': KEYWORDS
     }
 }

ℹ️ see the [list of Zenodo metadata](https://github.com/eamena-project/eamena-arches-dev/tree/main/data/bibref#metadata)

### Query the databae

Query the database API using the `GEOJSON_URL` URL selection. Create the 'data' JSON

In [30]:
resp = requests.get(GEOJSON_URL)
data = resp.json()

- If you are running this script on your local machine, uncomment/run the following code chunk

In [ ]:
# run only in local, change also paths
mydir = "C:\\Rprojects\\eamena-arches-dev\\dev\\citations"
os.chdir(mydir)
os.chdir(os.getcwd() + "/exports")
from pathlib import Path
Path(os.getcwd() + "/" + FILENAME).mkdir(parents = True, exist_ok = True)
os.chdir(os.getcwd() + "/" + FILENAME)

### Data output

Write JSON and ZIP locally

In [32]:

# JSON file name and ZIP file
json_file_name = FILENAME + ".geojson"
zip_file_name = FILENAME + ".zip"

# Create the JSON file and write the data to it
json_string = json.dumps(data, cls = NpEncoder)
json_string = json.loads(json_string)
with open(json_file_name, 'w') as json_file:
	json.dump(json_string, json_file, indent=4)
	print(json_file_name + " has been exported in " + os.getcwd())

# Create a ZIP file and add the JSON file to it
with zipfile.ZipFile(zip_file_name, "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(json_file_name)
    print(zip_file_name + " has been exported in " + os.getcwd())

data5.geojson has been exported in /content
data5.zip has been exported in /content


ℹ️ Further data can be created and files added into the ZIP

## Zenodo

### Creates an empty bucket

Paste your `ACCESS_TOKEN` (see Zenodo documentation [here](https://zenodo.org/account/settings/applications/tokens))

In [33]:
ACCESS_TOKEN = ''

Choose one of selected Zenodo deposit:
* Zenodo (`ZENODO_URL = 'https://zenodo.org/api/deposit/depositions'`)`
* Zenodo sanbox for tests (`ZENODO_URL = 'https://sandbox.zenodo.org/api/deposit/depositions'`)

In [34]:
ZENODO_URL = 'https://sandbox.zenodo.org/api/deposit/depositions'

Create the bucket

In [35]:
params = {'access_token': ACCESS_TOKEN}
r = requests.post(ZENODO_URL,
                   params=params,
                   json={})
r.status_code
r.json()
# collect the deposition id
deposition_id = r.json()['id']
print("The deposition_id is: " + str(deposition_id))

The deposition_id is: 10004658


### Add data

In [36]:
bucket_url = r.json()["links"]["bucket"]
with open(zip_file_name, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, zip_file_name),
        data = fp,
        params = params,
    )
r.json()

{'created': '2023-11-13T14:23:41.890489+00:00',
 'updated': '2023-11-13T14:23:42.781859+00:00',
 'version_id': 'c9a3ed44-805c-475f-ac3d-8df7b1df0679',
 'key': 'data5.zip',
 'size': 55881,
 'mimetype': 'application/zip',
 'checksum': 'md5:5ce84ec90f58a5dbf4e8f76baefbdcd8',
 'is_head': True,
 'delete_marker': False,
 'links': {'self': 'https://sandbox.zenodo.org/api/files/7b3eb1ef-8835-4641-9b71-d42f8723251a/data5.zip',
  'version': 'https://sandbox.zenodo.org/api/files/7b3eb1ef-8835-4641-9b71-d42f8723251a/data5.zip?versionId=c9a3ed44-805c-475f-ac3d-8df7b1df0679',
  'uploads': 'https://sandbox.zenodo.org/api/files/7b3eb1ef-8835-4641-9b71-d42f8723251a/data5.zip?uploads'}}

### Add metadata

In [37]:
r = requests.put('%s/%s' % (ZENODO_URL, deposition_id),
                  params = {'access_token': ACCESS_TOKEN},
                  data = json.dumps(metadata)) # ,
                  # headers = headers)
r.status_code
# 200

200

### Publish

In [38]:
r = requests.post('%s/%s/actions/publish' % (ZENODO_URL, deposition_id),
                      params={'access_token': ACCESS_TOKEN} )
r.status_code
# 504

202

### Check

Have a look at the last deposit (`r.json()[0]`)

In [39]:
r = requests.get(ZENODO_URL,
                  params={'access_token': ACCESS_TOKEN})
r.status_code
# 200
r.json()[0]

{'created': '2023-11-13T14:23:53.413048+00:00',
 'modified': '2023-11-13T14:23:54.698111+00:00',
 'id': 10004658,
 'conceptrecid': '10004657',
 'doi': '10.5072/zenodo.10004658',
 'conceptdoi': '10.5072/zenodo.10004657',
 'doi_url': 'https://doi.org/10.5072/zenodo.10004658',
 'metadata': {'title': 'data5',
  'doi': '10.5072/zenodo.10004658',
  'publication_date': '2023-11-13',
  'description': 'This is my first upload',
  'access_right': 'open',
  'creators': [{'name': 'EAMENA database',
    'affiliation': 'University of Oxford, University of Southampton'}],
  'keywords': ['Sistan', 'EAMENA'],
  'license': 'cc-by-4.0',
  'imprint_publisher': 'Zenodo',
  'upload_type': 'dataset',
  'prereserve_doi': {'doi': '10.5281/zenodo.10004658', 'recid': 10004658}},
 'title': 'data5',
 'links': {'self': 'https://sandbox.zenodo.org/api/records/10004658',
  'html': 'https://sandbox.zenodo.org/records/10004658',
  'doi': 'https://doi.org/10.5072/zenodo.10004658',
  'parent_doi': 'https://sandbox.zenodo

ℹ️ The Zenodo link to the record is recorded in `r.json()[0]['links']['html']`

In [45]:
!pip install jupyter_contrib_nbextensions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 35.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.1 MB/s eta 0:00:00
  Created wheel for jupyter_contrib_nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428780 sha256=6479691846491a4b76d4f5723a92b4639482b1630d87c6277023ff54ffa8b78e
  Stored in directory: /root/.cache/pip/wheels/ea/cc/7d/99ef154f984726b1201c0f72cfe9c9d7c5132c1a2ae4d8677f
  Created wheel for jupyter_contrib_core: filename=jupyter_contrib_core-0.4.2-py2.py3-none-any.whl size=17483 sha256=98dfd54d531c4a6f0615cd8ab09b61e6080f49036815dc2cded6075fbd0145e2
  Stored in directory: /root/.cache/pip/wheels/a9/52/88/e0643cdfd68f0562087918c37dd583378648dbc3df68b907f7
Successfully built jupyter_contrib_nbextensions jupyter_contrib_core


In [48]:
# show badge
from IPython.display import Markdown as md
badge = r.json()[0]['links']['badge']
md("![]({})".format(badge))

my badge ![](https://sandbox.zenodo.org/badge/doi/10.5072%2Fzenodo.10004658.svg)

In [40]:
r.json()[0]['links']['html']

'https://sandbox.zenodo.org/records/10004658'

ℹ️ Its DOI is `r.json()[0]['doi_url']`

In [41]:
r.json()[0]['doi_url']

'https://doi.org/10.5072/zenodo.10004658'